# Constants to discover where a statement is executed from

I hate the `__name__ == '__main__'`, it throws people off.  Like Y the F R U doing that.

This notebook creates exports three objects that will tell if:

* If the statement was called from a `__main__` module.
* If the statement was called interactively.
* If the statement was called on an imported object.

In [1]:
    __all__ = 'MAIN', 'INTERACTIVE', 'IMPORTED'

In [2]:
    from inspect import currentframe, getouterframes
    from pathlib import Path

The name of the file provides a baseline for comparing call frames.

In [3]:
    FILE = globals().get('__file__', None)

Get the current frame and retreive the first frame called outside of this module.

In [4]:
    class state:
        """Base class for the state objects."""
        def __bool__(self): 
            global FILE
            for frame in getouterframes(currentframe()):
                frame = frame.frame
                file = frame.f_globals.get('__file__', None)
                if file != FILE or FILE is None:
                    return frame.f_globals.get('__name__'), file, frame

## Is the module being run from the command line as an application?

In [5]:
class main(state):
    """Use MAIN to discover if a statement is invoked as a __main__ program.
    
    replaces
    
        __name__ == '__main__' and __import__('sys').argv == __file__
    """
    def __bool__(self):
        name, file, frame = super().__bool__()
        return name == '__main__' and file == __import__('sys').argv[0]
    
MAIN = main()

## Is the code executed by IPython or Jupyter?

In [6]:
class interactive(state):
    """Use INTERACTIVE to discover if a statement is invoked from IPython or Jupyter.
    
    replaces
        
        __name__ == '__main__' and globals().get('__file__', None) == None
    """
    def __bool__(self):
        name, file, frame = super().__bool__()
        return name == '__main__' and file is None
    
INTERACTIVE = interactive()

## Is the module simply being imported?

In [7]:
class imported(state):
    """Use IMPORTED to discover if a statement is invoked as an imported module.
    
    replaces: 
    
        __name__ != '__main__'
    """
    def __bool__(self):
        name, file, frame = super().__bool__()
        return name != '__main__'
    
IMPORTED = imported()

# Developer

In [8]:
    def test_imported():
        assert IMPORTED, "the logic is borked."

In [9]:
    if INTERACTIVE:
        __import__('pytest').main("-s 2018-07-21-whereami.ipynb".split())

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.5.1, py-1.5.3, pluggy-0.6.0
rootdir: C:\Users\deathbeds\deathbeds.github.io\deathbeds, inifile: tox.ini
plugins: testmon-0.9.12, remotedata-0.2.1, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2, importnb-0.4.1, hypothesis-3.66.1
collected 1 item

2018-07-21-whereami.ipynb .

========================== 1 passed in 0.33 seconds ===========================
